In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

### Importation des bibliothèques ###
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Utilisation du backend 'Agg' pour générer des images sans interface graphique
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import time
from numba import njit, prange  # Numba pour accélérer certaines fonctions numériques
import random as rand

### Paramètres par défaut ###
dx = 0.2  # Taille du pas spatial pour la discrétisation

# Paramètres par défaut pour les cellules
cell_params_default = {
    'c0': 1.0,        # Terme constant dans l'équation de R
    'a': 0.05,        # Force du couplage entre le signal et l'activateur A
    'gamma': 0.5,     # Taux de relaxation de R
    'Kd': 1e-5,       # Constante de dissociation pour le signal
    'sigma': 0.15,    # Intensité du bruit
    'epsilon': 0.2,   # Petit paramètre pour l'équation de R
    'cStim': 100.0,   # Concentration du stimulus externe
    'aPDE': 10.0,     # Taux de dégradation du cAMP
    'rho': 1.0,       # Facteur de production du cAMP
    'D': 1000.0,      # Taux de production du cAMP par les cellules activées
    'a0': 1.0,        # Production basale de cAMP
    'af': 0.0         # Seuil d'activation pour la production de cAMP
}

# Paramètres par défaut pour la grille
grid_params_default = {
    'dx': dx,
    'D_sig': 5.0,                   # Coefficient de diffusion du cAMP
    'box_size_x': 50.0,             # Taille de la boîte en x
    'box_size_y': 50.0,             # Taille de la boîte en y
    'agent_dim': 2 * dx,            # Dimension d'un agent (cellule)
    'num_agents': 200000             # Nombre total d'agents (cellules)
}

### Fonctions auxiliaires ###

@njit
def addcD(dt, laplacian_S, D_sig, aPDE, rho, a0, D, signal, A_grid_hsided):
    """
    Fonction pour mettre à jour le signal cAMP en tenant compte de la diffusion,
    de la dégradation et de la production par les cellules activées.
    """
    return dt * (D_sig * laplacian_S - aPDE * signal + rho * a0 + rho * D * A_grid_hsided)

@njit
def accumulate_arr(coord, arr, shape):
    """
    Fonction pour accumuler les valeurs des cellules sur une grille.
    """
    result = np.zeros(shape)
    for idx in range(len(arr)):
        x = coord[0, idx]
        y = coord[1, idx]
        result[x, y] += arr[idx]
    return result

@njit
def scale_up(A_small, k):
    """
    Fonction pour agrandir une matrice en la dupliquant k fois dans chaque dimension.
    """
    ny, nx = A_small.shape
    A_large = np.zeros((ny * k, nx * k))
    for i in range(ny):
        for j in range(nx):
            for di in range(k):
                for dj in range(k):
                    A_large[i * k + di, j * k + dj] = A_small[i, j]
    return A_large


### Classe de population de cellules signalant ###
class dictyPop:
    def __init__(self, T, save_every, g_params=grid_params_default, c_params=cell_params_default,
                 noise=True, progress_report=True, save_data=True):
        # Initialisation des paramètres
        self.g_params = g_params
        self.c_params = c_params
        self.T = T  # Temps total de simulation
        self.dt = self.g_params['dx'] ** 2 / (8 * self.g_params['D_sig'])  # Pas de temps estimé pour la stabilité
        self.Tsteps = int(self.T / self.dt)  # Nombre total de pas de temps
        self.signal_size = (
            int(self.g_params['box_size_x'] / self.g_params['dx']),
            int(self.g_params['box_size_y'] / self.g_params['dx'])
        )  # Taille de la grille de signal
        self.fluxes = np.zeros(4)  # Flux pour les conditions aux limites (non utilisé ici)
        self.p_r = progress_report  # Indicateur de rapport de progression
        self.save_every = save_every  # Fréquence de sauvegarde
        self.save_data = save_data  # Indicateur de sauvegarde des données
        self.noise_flag = noise  # Indicateur pour ajouter du bruit ou non

        self.num_agents = self.g_params['num_agents']  # Nombre de cellules
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])  # Ratio de dimension des agents
        self.agent_size = (
            self.signal_size[0] // self.agent_ratio,
            self.signal_size[1] // self.agent_ratio
        )  # Taille de la grille des agents


         # Initialisation des états des cellules
        self.A = np.random.normal(loc=-1.0, scale=0.1, size=self.num_agents)
        self.R = np.random.normal(loc=1.0, scale=0.1, size=self.num_agents)

        self.signal = np.zeros(self.signal_size, dtype=np.float64)

        # Positions des agents
        self.coord = np.zeros((2, self.num_agents), dtype=np.int64)
        for agent in range(self.num_agents):
            self.coord[0, agent] = rand.randint(0, self.agent_size[0] - 1)
            self.coord[1, agent] = rand.randint(0, self.agent_size[1] - 1)

        # Facteur d'interpolation
        self.interp_factor = self.agent_ratio


    def getAGrid(self):
        """
        Fonction pour obtenir la grille des valeurs de A accumulées sur la grille des agents.
        """
        A_grid = accumulate_arr(self.coord, self.A, self.agent_size)
        return A_grid

    def getRGrid(self):
        """
        Fonction pour obtenir la grille des valeurs de R accumulées sur la grille des agents.
        """
        R_grid = accumulate_arr(self.coord, self.R, self.agent_size)
        return R_grid

    def getSignalAtAgents(self):
        """
        Fonction pour obtenir le signal cAMP aux positions des agents.
        """
        S_small = self.signal[::self.interp_factor, ::self.interp_factor]  # Réduction de la grille du signal
        signal_at_agents = S_small[self.coord[0, :], self.coord[1, :]]  # Extraction du signal aux positions des agents
        return signal_at_agents

    @staticmethod
    @njit
    def update_cells(A, R, signal_at_agents, dt, a, Kd, gamma, c0, epsilon, sigma, noise_flag):
        """
        Fonction pour mettre à jour les états des cellules (A et R).
        """
        num_agents = A.shape[0]
        dA = np.zeros_like(A)
        dR = np.zeros_like(R)
        for i in range(num_agents):
            I_S = a * np.log1p(signal_at_agents[i] / Kd)  # Terme d'entrée du signal
            dA_i = (A[i] - (A[i] ** 3) / 3 - R[i] + I_S) * dt  # Équation de FitzHugh-Nagumo pour A
            if noise_flag:
                dA_i += sigma * np.sqrt(dt) * np.random.randn()  # Ajout du bruit si activé
            dR_i = (A[i] - gamma * R[i] + c0) * epsilon * dt  # Équation pour R
            A[i] += dA_i  # Mise à jour de A
            R[i] += dR_i  # Mise à jour de R
        return A, R

    @staticmethod
    @njit
    def compute_laplacian(S, dx):
        """
        Fonction pour calculer le Laplacien du signal S avec conditions périodiques.
        """
        ny, nx = S.shape
        laplacian_S = np.zeros_like(S)
        for i in range(ny):
            for j in range(nx):
                # Conditions périodiques en utilisant l'opérateur modulo
                up = S[(i - 1) % ny, j]
                down = S[(i + 1) % ny, j]
                left = S[i, (j - 1) % nx]
                right = S[i, (j + 1) % nx]
                laplacian_S[i, j] = (up + down + left + right - 4 * S[i, j]) / dx ** 2  # Calcul du Laplacien
        return laplacian_S

    def update(self):
        """
        Fonction principale pour mettre à jour l'état du système à chaque pas de temps.
        """
        # Mise à jour des cellules
        signal_at_agents = self.getSignalAtAgents()  # Obtention du signal aux positions des agents
        # Extraction des paramètres nécessaires
        a = self.c_params['a']
        Kd = self.c_params['Kd']
        gamma = self.c_params['gamma']
        c0 = self.c_params['c0']
        epsilon = self.c_params['epsilon']
        sigma = self.c_params['sigma']

        # Mise à jour des états des cellules
        self.A, self.R = self.update_cells(
            self.A, self.R, signal_at_agents, self.dt,
            a, Kd, gamma, c0, epsilon, sigma, self.noise_flag
        )

        # Calcul du A_grid et de A_grid_big pour correspondre à la taille de la grille du signal
        A_grid = self.getAGrid()
        A_grid_big = scale_up(A_grid, self.agent_ratio)

        # Calcul du Laplacien du signal
        laplacian_S = self.compute_laplacian(self.signal, self.g_params['dx'])

        # Calcul de la production du signal cAMP
        af = self.c_params['af']
        aPDE = self.c_params['aPDE']
        rho = self.c_params['rho']
        a0 = self.c_params['a0']
        D = self.c_params['D']
        D_sig = self.g_params['D_sig']

        # Application de la fonction Heaviside pour déterminer les zones actives
        A_grid_hsided = np.heaviside(A_grid_big - af, 0.5)
        # Mise à jour du signal cAMP
        self.signal += addcD(self.dt, laplacian_S, D_sig, aPDE, rho, a0, D, self.signal, A_grid_hsided)

        # S'assurer que le signal reste non négatif
        self.signal = np.maximum(self.signal, 0.0)

        return self.signal

### Exécution du script et production de séries temporelles de l'intensité du signal ###
def timeseries_a(a, t0, tf, freq, N):
    ### Paramètres de la cellule et de la grille ###
    # N  # Taille de la population
    cell_params = cell_params_default
    grid_params = grid_params_default
    grid_params['num_agents'] = N

    noise = True
    pop = dictyPop(T=tf, save_every=freq, c_params=cell_params, g_params=grid_params, noise=noise)
    pop.A = np.zeros(pop.A.shape)
    pop.R = np.zeros(pop.R.shape)

    size = 10  # Taille de la zone pour l'extraction des séries temporelles
    TimeSeries = []

    num_steps = int(tf / pop.dt)
    for i in range(num_steps):
        S = pop.update()
        if i % 5000 == 0:
            plt.figure()
            plt.imshow(S, origin='lower', extent=[0, pop.g_params['box_size_x'], 0, pop.g_params['box_size_y']])
            plt.colorbar(label='Concentration de signal')

            # Obtenir les positions des cellules en coordonnées physiques
            x_positions = pop.coord[1, :] * pop.g_params['agent_dim'] + pop.g_params['agent_dim'] / 2
            y_positions = pop.coord[0, :] * pop.g_params['agent_dim'] + pop.g_params['agent_dim'] / 2

            # Déterminer quelles cellules sont dans la boîte
            in_box = (
                (x_positions >= 0) & (x_positions <= pop.g_params['box_size_x']) &
                (y_positions >= 0) & (y_positions <= pop.g_params['box_size_y'])
            )
            num_cells_in_box = np.sum(in_box)

            # Tracer les positions des cellules à l'intérieur de la boîte
            plt.scatter(x_positions[in_box], y_positions[in_box], c='red', s=0.01)

            # Modifier le titre pour inclure le nombre de cellules
            plt.title(f"t={i * pop.dt:.2f}, N_cells={num_cells_in_box}")

            # Sauvegarde de la figure
            path = "'./images_aleatoires_03'"
            if not os.path.exists(path):
                os.makedirs(path)
            plt.savefig(f"{path}/{int(i * pop.dt)}.png")
            plt.close()

        if i % freq == 0:
            NewCol = []
            for ix in range(0, S.shape[0], size):
                for jy in range(0, S.shape[1], size):
                    region = S[ix:ix + size, jy:jy + size]
                    if region.size > 0:
                        mean_region = np.mean(region)
                        mean_S = np.mean(S)
                        if mean_S != 0:
                            NewCol.append(mean_region / mean_S)
                        else:
                            NewCol.append(0)
            TimeSeries.append(NewCol)

    TimeSeries = pd.DataFrame(TimeSeries)
    return TimeSeries

# Exemple d'exécution
if __name__ == "__main__":
    t0 = 0
    tf = 50000  # Temps final de simulation
    freq = 1000  # Fréquence d'extraction des séries temporelles
    a = None  # Non utilisé dans la fonction, mais inclus pour correspondre à la signature
    timeseries = timeseries_a(a, t0, tf, freq, N= grid_params_default['num_agents'])
    timeseries.to_csv('timeseries.csv', index=False)  # Sauvegarde des séries temporelles


In [21]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

### Importation des bibliothèques ###
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Utilisation du backend 'Agg' pour générer des images sans interface graphique
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import time
from numba import njit, prange  # Numba pour accélérer certaines fonctions numériques
import random as rand

### Paramètres par défaut ###
dx = 0.2  # Taille du pas spatial pour la discrétisation

# Paramètres par défaut pour les cellules
cell_params_default = {
    'c0': 1.0,        # Terme constant dans l'équation de R
    'a': 0.05,        # Force du couplage entre le signal et l'activateur A
    'gamma': 0.5,     # Taux de relaxation de R
    'Kd': 1e-5,       # Constante de dissociation pour le signal
    'sigma': 0.15,    # Intensité du bruit
    'epsilon': 0.2,   # Petit paramètre pour l'équation de R
    'cStim': 100.0,   # Concentration du stimulus externe
    'aPDE': 10.0,     # Taux de dégradation du cAMP
    'rho': 1.0,       # Facteur de production du cAMP
    'D': 1000.0,      # Taux de production du cAMP par les cellules activées
    'a0': 1.0,        # Production basale de cAMP
    'af': 0.0         # Seuil d'activation pour la production de cAMP
}

# Paramètres par défaut pour la grille
grid_params_default = {
    'dx': dx,
    'D_sig': 5.0,                   # Coefficient de diffusion du cAMP
    'box_size_x': 50.0,             # Taille de la boîte en x
    'box_size_y': 50.0,             # Taille de la boîte en y
    'agent_dim': 2 * dx,            # Dimension d'un agent (cellule)
    'num_agents': 200000            # Nombre total d'agents (cellules)
}

### Fonctions auxiliaires ###

@njit
def addcD(dt, laplacian_S, D_sig, aPDE, rho, a0, D, signal, A_grid_hsided):
    """
    Fonction pour mettre à jour le signal cAMP en tenant compte de la diffusion,
    de la dégradation et de la production par les cellules activées.
    """
    return dt * (D_sig * laplacian_S - aPDE * signal + rho * a0 + rho * D * A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    """
    Fonction pour accumuler les valeurs des cellules sur une grille en utilisant np.add.at.
    """
    result = np.zeros(shape, dtype=arr.dtype)
    np.add.at(result, (coord[0], coord[1]), arr)
    return result

### Classe de population de cellules signalant ###
class dictyPop:
    def __init__(self, T, save_every, g_params=grid_params_default, c_params=cell_params_default,
                 noise=True, progress_report=True, save_data=True):
        # Initialisation des paramètres
        self.g_params = g_params
        self.c_params = c_params
        self.T = T  # Temps total de simulation
        self.dt = self.g_params['dx'] ** 2 / (8 * self.g_params['D_sig'])  # Pas de temps estimé pour la stabilité
        self.Tsteps = int(self.T / self.dt)  # Nombre total de pas de temps
        self.signal_size = (
            int(self.g_params['box_size_x'] / self.g_params['dx']),
            int(self.g_params['box_size_y'] / self.g_params['dx'])
        )  # Taille de la grille de signal
        self.fluxes = np.zeros(4)  # Flux pour les conditions aux limites (non utilisé ici)
        self.p_r = progress_report  # Indicateur de rapport de progression
        self.save_every = save_every  # Fréquence de sauvegarde
        self.save_data = save_data  # Indicateur de sauvegarde des données
        self.noise_flag = noise  # Indicateur pour ajouter du bruit ou non

        self.num_agents = self.g_params['num_agents']  # Nombre de cellules
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])  # Ratio de dimension des agents
        self.agent_size = (
            self.signal_size[0] // self.agent_ratio,
            self.signal_size[1] // self.agent_ratio
        )  # Taille de la grille des agents

        # Initialisation des états des cellules
        self.A = np.random.normal(loc=-1.0, scale=0.1, size=self.num_agents).astype(np.float32)
        self.R = np.random.normal(loc=1.0, scale=0.1, size=self.num_agents).astype(np.float32)

        self.signal = np.zeros(self.signal_size, dtype=np.float32)

        # Positions des agents
        self.coord = np.zeros((2, self.num_agents), dtype=np.int64)
        self.coord[0, :] = np.random.randint(0, self.agent_size[0], size=self.num_agents)
        self.coord[1, :] = np.random.randint(0, self.agent_size[1], size=self.num_agents)

        # Facteur d'interpolation
        self.interp_factor = self.agent_ratio

    def getAGrid(self):
        """
        Fonction pour obtenir la grille des valeurs de A accumulées sur la grille des agents.
        """
        A_grid = accumulate_arr(self.coord, self.A, self.agent_size)
        return A_grid

    def getSignalAtAgents(self):
        """
        Fonction pour obtenir le signal cAMP aux positions des agents.
        """
        S_small = self.signal[::self.interp_factor, ::self.interp_factor]  # Réduction de la grille du signal
        signal_at_agents = S_small[self.coord[0, :], self.coord[1, :]]  # Extraction du signal aux positions des agents
        return signal_at_agents

    @staticmethod
    @njit
    def update_cells(A, R, signal_at_agents, dt, a, Kd, gamma, c0, epsilon, sigma, noise_flag):
        """
        Fonction vectorisée pour mettre à jour les états des cellules (A et R).
        """
        I_S = a * np.log1p(signal_at_agents / Kd)
        dA = (A - (A ** 3) / 3 - R + I_S) * dt
        if noise_flag:
            dA += sigma * np.sqrt(dt) * np.random.randn(A.size)
        dR = (A - gamma * R + c0) * epsilon * dt
        A += dA
        R += dR
        return A, R

    @staticmethod
    @njit(parallel=True)
    def compute_laplacian(S, dx):
        """
        Fonction pour calculer le Laplacien du signal S avec conditions périodiques.
        """
        ny, nx = S.shape
        laplacian_S = np.zeros_like(S)
        for i in prange(ny):
            for j in range(nx):
                # Conditions périodiques
                up = S[(i - 1) % ny, j]
                down = S[(i + 1) % ny, j]
                left = S[i, (j - 1) % nx]
                right = S[i, (j + 1) % nx]
                laplacian_S[i, j] = (up + down + left + right - 4 * S[i, j]) / dx ** 2
        return laplacian_S

    def update(self):
        """
        Fonction principale pour mettre à jour l'état du système à chaque pas de temps.
        """
        # Mise à jour des cellules
        signal_at_agents = self.getSignalAtAgents()  # Obtention du signal aux positions des agents
        # Extraction des paramètres nécessaires
        a = self.c_params['a']
        Kd = self.c_params['Kd']
        gamma = self.c_params['gamma']
        c0 = self.c_params['c0']
        epsilon = self.c_params['epsilon']
        sigma = self.c_params['sigma']

        # Mise à jour des états des cellules
        self.A, self.R = self.update_cells(
            self.A, self.R, signal_at_agents, self.dt,
            a, Kd, gamma, c0, epsilon, sigma, self.noise_flag
        )

        # Calcul du A_grid_hsided sans redimensionnement
        A_grid = self.getAGrid()
        A_grid_hsided = np.heaviside(A_grid - self.c_params['af'], 0.5)
        # Étendre A_grid_hsided à la taille de la grille du signal
        A_grid_hsided_big = np.kron(A_grid_hsided, np.ones((self.agent_ratio, self.agent_ratio)))

        # Calcul du Laplacien du signal
        laplacian_S = self.compute_laplacian(self.signal, self.g_params['dx'])

        # Calcul de la production du signal cAMP
        aPDE = self.c_params['aPDE']
        rho = self.c_params['rho']
        a0 = self.c_params['a0']
        D = self.c_params['D']
        D_sig = self.g_params['D_sig']

        # Mise à jour du signal cAMP
        self.signal += addcD(self.dt, laplacian_S, D_sig, aPDE, rho, a0, D, self.signal, A_grid_hsided_big)

        # S'assurer que le signal reste non négatif
        self.signal = np.maximum(self.signal, 0.0)

        return self.signal

### Exécution du script et production de séries temporelles de l'intensité du signal ###
def timeseries_a(a, t0, tf, freq, N):
    ### Paramètres de la cellule et de la grille ###
    cell_params = cell_params_default
    grid_params = grid_params_default
    grid_params['num_agents'] = N

    noise = True
    pop = dictyPop(T=tf, save_every=freq, c_params=cell_params, g_params=grid_params, noise=noise)
    pop.A = np.zeros(pop.A.shape, dtype=np.float32)
    pop.R = np.zeros(pop.R.shape, dtype=np.float32)

    size = 10  # Taille de la zone pour l'extraction des séries temporelles
    TimeSeries = []

    num_steps = int(tf / pop.dt)
    for i in range(num_steps):
        S = pop.update()
        if i % 5000 == 0:
            plt.figure()
            plt.imshow(S, origin='lower', extent=[0, pop.g_params['box_size_x'], 0, pop.g_params['box_size_y']])
            plt.colorbar(label='Concentration de signal')

            # Obtenir les positions des cellules en coordonnées physiques
            x_positions = pop.coord[1, :] * pop.g_params['agent_dim'] + pop.g_params['agent_dim'] / 2
            y_positions = pop.coord[0, :] * pop.g_params['agent_dim'] + pop.g_params['agent_dim'] / 2

            # Tracer les positions des cellules
            plt.scatter(x_positions, y_positions, c='red', s=0.01)

            # Modifier le titre pour inclure le temps
            plt.title(f"t={i * pop.dt:.2f}")

            # Sauvegarde de la figure
            path = "./images_optimisees"
            if not os.path.exists(path):
                os.makedirs(path)
            plt.savefig(f"{path}/{int(i * pop.dt)}.png")
            plt.close()

        if i % freq == 0:
            NewCol = []
            for ix in range(0, S.shape[0], size):
                for jy in range(0, S.shape[1], size):
                    region = S[ix:ix + size, jy:jy + size]
                    if region.size > 0:
                        mean_region = np.mean(region)
                        mean_S = np.mean(S)
                        if mean_S != 0:
                            NewCol.append(mean_region / mean_S)
                        else:
                            NewCol.append(0)
            TimeSeries.append(NewCol)

    TimeSeries = pd.DataFrame(TimeSeries)
    return TimeSeries

# Exemple d'exécution
if __name__ == "__main__":
    t0 = 0
    tf = 50000  # Temps final de simulation
    freq = 1000  # Fréquence d'extraction des séries temporelles
    a = None  # Non utilisé dans la fonction, mais inclus pour correspondre à la signature
    timeseries = timeseries_a(a, t0, tf, freq, N=grid_params_default['num_agents'])
    timeseries.to_csv('timeseries.csv', index=False)  # Sauvegarde des séries temporelles


KeyboardInterrupt: 

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

### Importation des bibliothèques ###
import numpy as np
import matplotlib
matplotlib.use("Agg")  # Utilisation du backend 'Agg' pour générer des images sans interface graphique
import matplotlib.pyplot as plt
import os
import sys
import pandas as pd
import time
from numba import njit, prange  # Numba pour accélérer certaines fonctions numériques
import random as rand

### Paramètres par défaut ###
dx = 0.5  # Taille du pas spatial pour la discrétisation

# Paramètres par défaut pour les cellules
cell_params_default = {
    'c0': 1.0,        # Terme constant dans l'équation de R
    'a': 0.05,        # Force du couplage entre le signal et l'activateur A
    'gamma': 0.5,     # Taux de relaxation de R
    'Kd': 1e-5,       # Constante de dissociation pour le signal
    'sigma': 0.15,    # Intensité du bruit
    'epsilon': 0.2,   # Petit paramètre pour l'équation de R
    'cStim': 100.0,   # Concentration du stimulus externe
    'aPDE': 10.0,     # Taux de dégradation du cAMP
    'rho': 1.0,       # Facteur de production du cAMP
    'D': 1000.0,      # Taux de production du cAMP par les cellules activées
    'a0': 1.0,        # Production basale de cAMP
    'af': 0.0         # Seuil d'activation pour la production de cAMP
}

# Paramètres par défaut pour la grille
grid_params_default = {
    'dx': dx,
    'D_sig': 5.0,                   # Coefficient de diffusion du cAMP
    'box_size_x': 50.0,             # Taille de la boîte en x
    'box_size_y': 50.0,             # Taille de la boîte en y
    'agent_dim': 2 * dx,            # Dimension d'un agent (cellule)
    'num_agents': 15000            # Nombre total d'agents (cellules)
}

### Fonctions auxiliaires ###

@njit
def addcD(dt, laplacian_S, D_sig, aPDE, rho, a0, D, signal, A_grid_hsided):
    """
    Fonction pour mettre à jour le signal cAMP en tenant compte de la diffusion,
    de la dégradation et de la production par les cellules activées.
    """
    return dt * (D_sig * laplacian_S - aPDE * signal + rho * a0 + rho * D * A_grid_hsided)

def accumulate_arr(coord, arr, shape):
    """
    Fonction pour accumuler les valeurs des cellules sur une grille en utilisant np.add.at.
    """
    result = np.zeros(shape, dtype=arr.dtype)
    np.add.at(result, (coord[0], coord[1]), arr)
    return result

### Classe de population de cellules signalant ###
class dictyPop:
    def __init__(self, T, save_every, g_params=grid_params_default, c_params=cell_params_default,
                 noise=True, progress_report=True, save_data=True):
        # Initialisation des paramètres
        self.g_params = g_params
        self.c_params = c_params
        self.T = T  # Temps total de simulation
        self.dt = self.g_params['dx'] ** 2 / (8 * self.g_params['D_sig'])  # Pas de temps estimé pour la stabilité
        self.Tsteps = int(self.T / self.dt)  # Nombre total de pas de temps
        self.signal_size = (
            int(self.g_params['box_size_x'] / self.g_params['dx']),
            int(self.g_params['box_size_y'] / self.g_params['dx'])
        )  # Taille de la grille de signal
        self.fluxes = np.zeros(4)  # Flux pour les conditions aux limites (non utilisé ici)
        self.p_r = progress_report  # Indicateur de rapport de progression
        self.save_every = save_every  # Fréquence de sauvegarde
        self.save_data = save_data  # Indicateur de sauvegarde des données
        self.noise_flag = noise  # Indicateur pour ajouter du bruit ou non

        self.num_agents = self.g_params['num_agents']  # Nombre de cellules
        self.agent_ratio = int(self.g_params['agent_dim'] / self.g_params['dx'])  # Ratio de dimension des agents
        self.agent_size = (
            self.signal_size[0] // self.agent_ratio,
            self.signal_size[1] // self.agent_ratio
        )  # Taille de la grille des agents

        # Initialisation des états des cellules
        self.A = np.random.normal(loc=-1.0, scale=0.1, size=self.num_agents).astype(np.float32)
        self.R = np.random.normal(loc=1.0, scale=0.1, size=self.num_agents).astype(np.float32)

        self.signal = np.zeros(self.signal_size, dtype=np.float32)

        # Positions des agents
        self.coord = np.zeros((2, self.num_agents), dtype=np.int64)
        self.coord[0, :] = np.random.randint(0, self.agent_size[0], size=self.num_agents)
        self.coord[1, :] = np.random.randint(0, self.agent_size[1], size=self.num_agents)

        # Facteur d'interpolation
        self.interp_factor = self.agent_ratio

    def getAGrid(self):
        """
        Fonction pour obtenir la grille des valeurs de A accumulées sur la grille des agents.
        """
        A_grid = accumulate_arr(self.coord, self.A, self.agent_size)
        return A_grid

    def getSignalAtAgents(self):
        """
        Fonction pour obtenir le signal cAMP aux positions des agents.
        """
        S_small = self.signal[::self.interp_factor, ::self.interp_factor]  # Réduction de la grille du signal
        signal_at_agents = S_small[self.coord[0, :], self.coord[1, :]]  # Extraction du signal aux positions des agents
        return signal_at_agents

    @staticmethod
    @njit
    def update_cells(A, R, signal_at_agents, dt, a, Kd, gamma, c0, epsilon, sigma, noise_flag):
        """
        Fonction vectorisée pour mettre à jour les états des cellules (A et R).
        """
        I_S = a * np.log1p(signal_at_agents / Kd)
        dA = (A - (A ** 3) / 3 - R + I_S) * dt
        if noise_flag:
            dA += sigma * np.sqrt(dt) * np.random.randn(A.size)
        dR = (A - gamma * R + c0) * epsilon * dt
        A += dA
        R += dR
        return A, R

    @staticmethod
    @njit(parallel=True)
    def compute_laplacian(S, dx):
        """
        Fonction pour calculer le Laplacien du signal S avec conditions de Neumann (flux nul).
        """
        ny, nx = S.shape
        laplacian_S = np.zeros_like(S)
        
        for i in prange(ny):
            for j in range(nx):
                if i == 0:  # Bord supérieur (Neumann)
                    up = S[i, j]  # Même valeur pour le bord
                else:
                    up = S[i - 1, j]
                
                if i == ny - 1:  # Bord inférieur (Neumann)
                    down = S[i, j]  # Même valeur pour le bord
                else:
                    down = S[i + 1, j]
                
                if j == 0:  # Bord gauche (Neumann)
                    left = S[i, j]  # Même valeur pour le bord
                else:
                    left = S[i, j - 1]
                
                if j == nx - 1:  # Bord droit (Neumann)
                    right = S[i, j]  # Même valeur pour le bord
                else:
                    right = S[i, j + 1]
                
                laplacian_S[i, j] = (up + down + left + right - 4 * S[i, j]) / dx ** 2

        return laplacian_S

    # Ancienne version avec conditions périodiques (commentée) :
    """
    @staticmethod
    @njit(parallel=True)
    def compute_laplacian(S, dx):
        Fonction pour calculer le Laplacien du signal S avec conditions périodiques.
        ny, nx = S.shape
        laplacian_S = np.zeros_like(S)
        for i in prange(ny):
            for j in range(nx):
                # Conditions périodiques
                up = S[(i - 1) % ny, j]
                down = S[(i + 1) % ny, j]
                left = S[i, (j - 1) % nx]
                right = S[i, (j + 1) % nx]
                laplacian_S[i, j] = (up + down + left + right - 4 * S[i, j]) / dx ** 2
        return laplacian_S
    """

    def update(self):
        """
        Fonction principale pour mettre à jour l'état du système à chaque pas de temps.
        """
        # Mise à jour des cellules
        signal_at_agents = self.getSignalAtAgents()  # Obtention du signal aux positions des agents
        # Extraction des paramètres nécessaires
        a = self.c_params['a']
        Kd = self.c_params['Kd']
        gamma = self.c_params['gamma']
        c0 = self.c_params['c0']
        epsilon = self.c_params['epsilon']
        sigma = self.c_params['sigma']

        # Mise à jour des états des cellules
        self.A, self.R = self.update_cells(
            self.A, self.R, signal_at_agents, self.dt,
            a, Kd, gamma, c0, epsilon, sigma, self.noise_flag
        )

        # Calcul du A_grid_hsided sans redimensionnement
        A_grid = self.getAGrid()
        A_grid_hsided = np.heaviside(A_grid - self.c_params['af'], 0.5)
        # Étendre A_grid_hsided à la taille de la grille du signal
        A_grid_hsided_big = np.kron(A_grid_hsided, np.ones((self.agent_ratio, self.agent_ratio)))

        # Calcul du Laplacien du signal
        laplacian_S = self.compute_laplacian(self.signal, self.g_params['dx'])

        # Calcul de la production du signal cAMP
        aPDE = self.c_params['aPDE']
        rho = self.c_params['rho']
        a0 = self.c_params['a0']
        D = self.c_params['D']
        D_sig = self.g_params['D_sig']

        # Mise à jour du signal cAMP
        self.signal += addcD(self.dt, laplacian_S, D_sig, aPDE, rho, a0, D, self.signal, A_grid_hsided_big)

        # S'assurer que le signal reste non négatif
        self.signal = np.maximum(self.signal, 0.0)

        return self.signal

### Exécution du script et production de séries temporelles de l'intensité du signal ###
def timeseries_a(a, t0, tf, freq, N):
    ### Paramètres de la cellule et de la grille ###
    cell_params = cell_params_default
    grid_params = grid_params_default
    grid_params['num_agents'] = N

    noise = True
    pop = dictyPop(T=tf, save_every=freq, c_params=cell_params, g_params=grid_params, noise=noise)
    pop.A = np.zeros(pop.A.shape, dtype=np.float32)
    pop.R = np.zeros(pop.R.shape, dtype=np.float32)

    size = 10  # Taille de la zone pour l'extraction des séries temporelles
    TimeSeries = []

    num_steps = int(tf / pop.dt)
    for i in range(num_steps):
        S = pop.update()
        if i % 5000 == 0:
            plt.figure()
            plt.imshow(S, origin='lower', extent=[0, pop.g_params['box_size_x'], 0, pop.g_params['box_size_y']])
            plt.colorbar(label='Concentration de signal')

            # Obtenir les positions des cellules en coordonnées physiques
            x_positions = pop.coord[1, :] * pop.g_params['agent_dim'] + pop.g_params['agent_dim'] / 2
            y_positions = pop.coord[0, :] * pop.g_params['agent_dim'] + pop.g_params['agent_dim'] / 2

            # Tracer les positions des cellules
            plt.scatter(x_positions, y_positions, c='red', s=0.01)

            # Modifier le titre pour inclure le temps
            plt.title(f"t={i * pop.dt:.2f}")

            # Sauvegarde de la figure
            path = "./images_optimisees_2"
            if not os.path.exists(path):
                os.makedirs(path)
            plt.savefig(f"{path}/{int(i * pop.dt)}.png")
            plt.close()

        if i % freq == 0:
            NewCol = []
            for ix in range(0, S.shape[0], size):
                for jy in range(0, S.shape[1], size):
                    region = S[ix:ix + size, jy:jy + size]
                    if region.size > 0:
                        mean_region = np.mean(region)
                        mean_S = np.mean(S)
                        if mean_S != 0:
                            NewCol.append(mean_region / mean_S)
                        else:
                            NewCol.append(0)
            TimeSeries.append(NewCol)

    TimeSeries = pd.DataFrame(TimeSeries)
    return TimeSeries

# Exemple d'exécution
if __name__ == "__main__":
    t0 = 0
    tf = 50000  # Temps final de simulation
    freq = 100  # Fréquence d'extraction des séries temporelles
    a = None  # Non utilisé dans la fonction, mais inclus pour correspondre à la signature
    timeseries = timeseries_a(a, t0, tf, freq, N=grid_params_default['num_agents'])
    timeseries.to_csv('timeseries.csv', index=False)  # Sauvegarde des séries temporelles
